In [7]:
import sys
'geopandas' in sys.modules

False

In [16]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from FeatureEngineering import create_time_matrix
from MatchingAlgorithm import run_algorithm

import MatchingAlgorithm 
import RouteVisualisation

import importlib 
importlib.reload(MatchingAlgorithm)


<module 'MatchingAlgorithm' from 'c:\\Users\\XieLo\\OneDrive\\Documents\\NUS\\NUS_ACADEMICS\\YEAR 4\\Sem 2\\BT4103 Capstone\\Github\\Capstone-Project\\MatchingAlgorithm.py'>

In [17]:
import FeatureEngineering

### Running algorithm

In [23]:
# INPUT YOUR API KEY

API_key = ''

In [19]:
orders_df = pd.read_csv("Simulated Data/order_data_1576.csv")
catchments_df = pd.read_csv("Simulated Data/catchment_data_1576.csv")
phlebs_df = pd.read_csv("Simulated Data/phleb_data_1576.csv")

In [20]:
orders_df = orders_df.iloc[:10]
phlebs_df = phlebs_df.iloc[:2]
catchments_df = catchments_df.iloc[:1] #remove this for multi-ends 

In [24]:
output = MatchingAlgorithm.run_algorithm(orders_df, catchments_df, phlebs_df, API_key)
print(output)

AttributeError: module 'MatchingAlgorithm' has no attribute 'run_algorithm_test'

### Visualisation

In [21]:
addresses_list = FeatureEngineering.get_coordinates_list(orders_df, catchments_df, phlebs_df)

In [22]:
import geopandas as gpd

json_result = json.loads(output)

polygon = gpd.read_file("Simulation\Gurugram_sample_Polygon.geojson")
addresses_list = addresses_list

RouteVisualisation.visualise_routes(json_result, polygon, addresses_list)

NameError: name 'output' is not defined